## Holoviz Panel and Xarray 5d plotting (limited 3d support)
This cell imports  libraries

In [ ]:
import xarray as xr
import panel as pn
import os
import holoviews as hv
from panel.interact import fixed
import numpy as np
hv.extension('bokeh')

This cell defines the plotting function

In [ ]:
h2 = pn.pane.Markdown('''## Plot''')
def getRange(dim,ds):
    return (ds.coords[dim].values.min(),ds.coords[dim].values.max())
def plot(filename):
    extension = filename.split('.')[1]
    if(extension == "3nc"):
        ds = xr.open_dataset(filename)
        def plot2(wavelength):
            return hv.HeatMap(ds.isel(wavelength=wavelength)).opts(cmap = 'Reds',width = 900,colorbar = True)
        dmap = hv.DynamicMap(plot2, kdims = ['wavelength'])
        return pn.Column(h2,pn.panel(dmap.redim.values(wavelength=ds.coords['wavelength'].values)))
    elif(extension == "5nc"):
        ds = xr.open_dataset(filename, chunks = {'Orientation':1})
        def plot(Orientation,Wavelength):
            polys = hv.Polygons([]).opts(fill_alpha=0.2, line_color='white')
            box_stream = hv.streams.BoxEdit(source=polys,num_objects = 1)
            def plot3(data, Orientation=Orientation):
                if not data or not any(len(d) for d in data.values()):
                    output = np.log(ds.isel(Orientation=Orientation).mean(dim = ['x','y']))
                    return hv.Image(output).opts(cmap = 'Blues',width = 900,colorbar = True,title=(f" Orientation: {Orientation}"))
                output = np.log(ds.isel(Orientation=Orientation,x=range(round(data['x0'][0]),round(data['x1'][0])),y = range(round(data['y0'][0]),round(data['y1'][0]))).mean(dim = ['x','y']))
                return hv.Image(output).opts(cmap = 'Reds',width = 900,colorbar = True,title=(f"Orientation: {Orientation}"))
            def plot2(Orientation, wavelength):
                data = ds.isel(Orientation=Orientation,wavelength = wavelength).mean(dim = 'Polarization')
                output = hv.HeatMap(data).opts(colorbar = True,title=(f"Wavelength: {wavelength}, Orientation: {Orientation}"))
                return output.redim(x=hv.Dimension('x', range= getRange('x',ds)),y=hv.Dimension('y', range= getRange('y',ds)))
            dmap = hv.DynamicMap(plot3, streams=[box_stream])
            return plot2(Orientation,Wavelength)*polys + dmap
        layout = pn.interact(plot, Orientation = ds.coords['Orientation'].values,Wavelength = ds.coords['wavelength'].values)
        return pn.Column(h2,layout[0],layout[1])
    else:
        return "<br>\n#INVALID FILE TYPE"

This cell creates the GUI

In [ ]:
h1 = pn.pane.Markdown('''
#Multidimensional spectral data visualizer
##Choose a file below
''')
layout = pn.interact(plot,filename=(["converted/"+file for file in os.listdir('converted')]))
GUI = pn.Row(pn.Column(h1,layout[0]),layout[1])
GUI.servable()